# FP - TRUN

In [5]:
from datetime import datetime
import mRun as mr
import utils_data as md
import numpy as np
import pandas as pd

def get_models(type):
    if   type == "FRFLO":
        return [
            { 'dt':'C2',  "e":100, "lr":0.001, "h":[100 , 100], "spn": 5000, "pe": [], "pt": []  },
            { 'dt':'C4',  "e":100, "lr":0.001, "h":[100 , 100], "spn": 5000, "pe": [], "pt": []  },
            { 'dt':'C1',  "e":100, "lr":0.001, "h":[100 , 100], "spn": 5000, "pe": [], "pt": []  },
        ]
    elif type == "FRALL1":
        return [
            { 'dt':'C2',  "e":20, "lr":0.001, "h":[100 , 100], "spn": 10000, "pe": [], "pt": []  },
            { 'dt':'C4',  "e":50, "lr":0.001, "h":[100 , 100], "spn": 10000, "pe": [], "pt": []  },
            { 'dt':'C1',  "e":50, "lr":0.001, "h":[100 , 100], "spn": 10000, "pe": [], "pt": []  },
            # { 'dt':'C0',  "e":100, "lr":0.001, "h":[100 , 100], "spn": 10000, "pe": [], "pt": []  },
        ]
    elif type == "FLALL":
            return [
            { 'dt':'C2',  "e":100, "lr":0.0001, "h":[100 , 100], "spn": 10000, "pe": [], "pt": []  },
            { 'dt':'C4',  "e":100, "lr":0.0001, "h":[100 , 100], "spn": 10000, "pe": [], "pt": []  },
            { 'dt':'C1',  "e":100, "lr":0.0001, "h":[100 , 100], "spn": 10000, "pe": [], "pt": []  },
            # { 'dt':'C0',  "e":100, "lr":0.001, "h":[100 , 100], "spn": 10000, "pe": [], "pt": []  },
        ]
    else: return []

In [11]:
def download_pandas( ):
    if rec_test: tsd.to_csv(md.LOGDAT + "ttestDS.csv"); print("\nDownload pandas test")
    if rec_eval: tse.to_csv(md.LOGDAT + "tevalDS.csv"); print("\nDownload pandas eval")
    return

def record_data( ex, dsp, dspo, type = "pt"  ):
    # print(len(np.array([str(xi[0]) for xi in ex["pt"][1]])));     print(len(tsd.columns))
    # tsd.insert(len(tsd.columns), ex["dt"] + 'FP_P', dsp2["FP_P"].map(lambda x: dc( x ) )    )
    #tsd[ "_PRED"] = np.array([str(xi[0]) for xi in ex["pt"][1]])  # pandas warnings! 
    dsp.insert(len(dsp.columns), ex["dt"] + '_FP_P',  dspo["FP_P"].map(lambda x: md.dc( x ) ))    
    dsp.insert(len(dsp.columns), ex["dt"] + '_PRDU',  np.array([int(xi[0])   for xi in ex[type][1]])  )    
    dsp.insert(len(dsp.columns), ex["dt"] + '_PRED',  np.array([str(xi)      for xi in ex[type][1]])  )    
    dsp.insert(len(dsp.columns), ex["dt"] + '_PROB',  np.array([str(xi)      for xi in ex[type][0]])  )    
    # error calc: 
    if ex["dt"] == "C1":
        dsp.insert(len(tsd.columns), ex["dt"] + '_ERR', 
                #( dsp.loc[:, ex["dt"] + '_FP_P'] == dsp.loc[:,ex["dt"] +'_PRDU']  )   )  
                np.array([ abs(dsp.loc[i,ex["dt"] +'_PRDU']-dsp.loc[i,ex["dt"] +'_FP_P'])>3 for i in range(len(dsp))  ])  )   
    else:
        dsp.insert(len(tsd.columns), ex["dt"] + '_ERR', ( dsp.loc[:, ex["dt"] + '_FP_P'] == dsp.loc[:,ex["dt"] +'_PRDU']  )  )   
    return dsp


#  logic

In [6]:
print("___Start!___" +  datetime.now().strftime('%H:%M:%S')  )
md.DESC = "FRALL1";  # FRFLO   FRALL1    FLALL
md.setDESC(md.DESC)
ALL_DS = md.LOGDAT + md.DESC + md.DSC 

# DATA READ  ------------------------------------------------ 
# md.mainRead2(ALL_DS, 1, 2 ) # , all = True, shuffle = True  ) 
md.mainRead2(path=ALL_DS, part=1, batch_size=2 ) # For testing I am forced to used JSON - column names and order may be different! 

url_test = md.LOGDAT + "FREXP1/" ; # url_test = "url"
force = False; excel = True  # dataFile = "frall2_json.txt"; labelFile = "datal.csv"     
md.get_tests(url_test, force, excel )

mr.final = "_" #_ _101

___Start!___18:40:30
set desc!
data read - 45822 - time:46.29591226577759


In [9]:
rec_eval = True
rec_test = False
tsd       = pd.DataFrame()
tse       = pd.DataFrame()

# OPERATIONS  ------------------------------------------------ 
md.get_columns(force=force, pp_excel=True)
if rec_test: tsd = md.dsp[["M", "FP"]]
if rec_eval: tse = md.dst.iloc[:md.spn, :2]

execc = get_models(md.DESC)


In [12]:
for ex in execc:
    md.spn = ex["spn"]; md.dType = ex["dt"]; mr.epochs = ex["e"]; mr.lr = ex["lr"]; mr.h = ex["h"]

    md.normalize()  
    mr.ninp, mr.nout, mr.top_k = md.getnn()
    md.MODEL = mr.get_hpar(mr.epochs, final=mr.final)
    md.MODEL_DIR = md.LOGDIR + md.DESC + '/' +  md.MODEL #+"/" 
    mr.model_path = md.MODEL_DIR + "/model.ckpt" 

    mr.build_network3()                                                                                                                                                                                                                                                                                    
    print(mr.model_path) 
    mr.clean_traina()

    # mr.train(it= ex["e"], disp=True, batch_size = 128, compt = True)
    ex["pe"] = mr.evaluate( )
    mr.vis_chart( ) # visualize the training chart

    ex["pt"] = mr.tests(url_test, p_col=False  )
    if rec_test: tsd = record_data( ex, tsd, md.dsp, type = "pt")    
    if rec_eval: tse = record_data( ex, tse, md.dst.iloc[:md.spn, :3], type = "pe")    

# PRINTING  ------------------------------------------------ 
print("end!___" +  datetime.now().strftime('%H:%M:%S')  )


build network
../../dmodels/FRALL1/slr_1E-03_NN2385x100x100x2_ep20_/model.ckpt
_____EVALUATION...
Model restored from file: ../../dmodels/FRALL1/slr_1E-03_NN2385x100x100x2_ep20_/model.ckpt
INFO:tensorflow:Restoring parameters from ../../dmodels/FRALL1/slr_1E-03_NN2385x100x100x2_ep20_/model.ckpt
Training   Accuracy: 0.989
Evaluation Accuracy: 0.990
Preview the first predictions:
RealVal: 1  - PP value: 1
RealVal: 1  - PP value: 1
RealVal: 1  - PP value: 1
RealVal: 1  - PP value: 1
RealVal: 1  - PP value: 1
RealVal: 1  - PP value: 1
RealVal: 1  - PP value: 1
RealVal: 1  - PP value: 1
RealVal: 1  - PP value: 1
RealVal: 1  - PP value: 1
RealVal: 1  - PP value: 1
RealVal: 1  - PP value: 1
RealVal: 1  - PP value: 1
RealVal: 1  - PP value: 1
RealVal: 1  - PP value: 1
RealVal: 1  - PP value: 1
RealVal: 1  - PP value: 1
RealVal: 1  - PP value: 1
RealVal: 0  - PP value: 0
RealVal: 1  - PP value: 1
denormalization all Evaluation : 10000 = 10000
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
Total

Confusion matrix, without normalization
end!___18:44:21

Download pandas eval


In [ ]:
# print_results(execc, typ = "pt") 
# DOWNLOAD ------------------------------------------------- 
download_pandas( )

# test 

In [14]:
# tse.head()


# Print formula 

In [22]:
# md.print_form2(md.dsp.iloc[0])
# PBV11270AD
# md.print_form2(md.dst[md.dst["M"]==100456].iloc[0])
md.print_form2(md.dsp[md.dsp["M"]=='PBV11270AD'].iloc[0])

astr = md.get_components(md.dsp.iloc[0])
# print(astr)

def print_list(astr, name = "/_logs_tr3" ):
    file = md.LOGDAT +  name + ".txt" # + md.DESC +
    f = open(file, 'w')
    for line in astr:
        f.write(line + "\n") 
    f.close()       #close file
    
print_list(astr)

N. of components: 48
components:____ tot ______FP_____values
M       -      (    )--(     ) - PBV11270AD
FP      -      (    )--(     ) -         59
100419  -      ( 176)--( 40.0) -  4.4444446% 		X
103011  -      (6116)--( 72.0) -  8.3333336% 		
103090  -      ( 186)--( 89.0) -  0.6666667% 		
103156  -      (2180)--(101.0) - 1.1111111000000002% 		
103247  -      (3316)--(101.0) - 0.5555555999999999% 		
103605  -      ( 960)--( 98.0) - 0.5555555999999999% 		
103797  -      (2411)--( 52.0) - 3.8888890000000003% 		X
103798  -      (3151)--( 82.0) - 0.11111110000000002% 		
103799  -      (4944)--( 87.0) - 13.888889599999999% 		
103933  -      ( 527)--( 88.0) - 0.055555600000000004% 		
105169  -      (2496)--( 41.0) -  0.3333333% 		X
105461  -      ( 584)--( 78.0) - 0.5555555999999999% 		
113554  -      (5645)--(151.0) -      0.066% 		
130077  -      (3131)--(116.0) - 0.055555600000000004% 		
130337  -      ( 155)--( 85.0) - 0.055555600000000004% 		
130659  -      ( 695)--(134.0) - 1.111111